In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\it272\OneDrive\Documents\anamoly-detection-system\data\raw\creditcard.csv")
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
from sklearn.preprocessing import StandardScaler
import numpy as np

scaler = StandardScaler()
df[['Amount_scaled']] = scaler.fit_transform(df[['Amount']])

z_scores = np.abs((df['Amount_scaled'] - df['Amount_scaled'].mean()) / df['Amount_scaled'].std())
df['z_anomaly'] = (z_scores > 3).astype(int)


In [3]:
Q1 = df['Amount'].quantile(0.25)
Q3 = df['Amount'].quantile(0.75)
IQR = Q3 - Q1

df['iqr_anomaly'] = (
    (df['Amount'] < (Q1 - 1.5 * IQR)) |
    (df['Amount'] > (Q3 + 1.5 * IQR))
).astype(int)


In [4]:
from sklearn.ensemble import IsolationForest

model = IsolationForest(contamination=0.001, random_state=42)
df['iforest_anomaly'] = model.fit_predict(df[['Amount']])
df['iforest_anomaly'] = df['iforest_anomaly'].map({1: 0, -1: 1})


In [5]:
df['final_anomaly_score'] = (
    df['z_anomaly'] +
    df['iqr_anomaly'] +
    df['iforest_anomaly']
)

df[['z_anomaly','iqr_anomaly','iforest_anomaly','final_anomaly_score']].head()


,z_anomaly,iqr_anomaly,iforest_anomaly,final_anomaly_score
0,0,0,0,0
1,0,0,0,0
2,0,1,0,1
3,0,0,0,0
4,0,0,0,0


In [6]:
def alert_level(score):
    if score == 3:
        return "HIGH"
    elif score == 2:
        return "MEDIUM"
    elif score == 1:
        return "LOW"
    else:
        return "NORMAL"

df['alert_level'] = df['final_anomaly_score'].apply(alert_level)


In [7]:
df['alert_level'].value_counts()


alert_level
NORMAL    252903
LOW        27828
MEDIUM      3880
HIGH         196
Name: count, dtype: int64

In [8]:
pd.crosstab(df['Class'], df['alert_level'])


alert_level,HIGH,LOW,MEDIUM,NORMAL
Class,,,,
0,196,27748,3869,252502
1,0,80,11,401
